# 프로젝트 명 : Seoul_Crime

# 1. 프로젝트 개요

In [173]:
import socket

hostname = socket.gethostname()

## 2. 데이터 개요
### 1) 데이터 읽기

In [174]:
import pandas as pd
import numpy as np
crime_raw_data=pd.DataFrame()

if "Dongmin" in hostname:
    crime_raw_data=pd.read_csv("../../data/02.Seoul_Crime/02. crime_in_Seoul.csv",thousands=",",encoding="euc-kr")
else :
    crime_raw_data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/02.Seoul_Crime/02. crime_in_Seoul.csv",thousands=",",encoding="euc-kr")

crime_raw_data.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,2.0
1,중부,살인,검거,2.0
2,중부,강도,발생,3.0
3,중부,강도,검거,3.0
4,중부,강간,발생,141.0


### 2) 데이터 개요 보기
- 문제점 :
  - RangeIndex : 65534
  - Not-null 데이터 : 310
  - null 데이터가 많음

In [175]:
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65534 entries, 0 to 65533
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


### 3) 잘못된 데이터 보기
- 죄종에 어떤 값이 들어가있는지 보기
- null 값에 얼마나 많은 데이터들이 있는지 보기

In [176]:
crime_raw_data["죄종"].unique()

array(['살인', '강도', '강간', '절도', '폭력', nan], dtype=object)

- 310번부터 65534번까지 데이터가 모두 NaN 값임

In [177]:
crime_raw_data[crime_raw_data["죄종"].isnull()]

,구분,죄종,발생검거,건수
310,NaN,NaN,NaN,NaN
311,NaN,NaN,NaN,NaN
312,NaN,NaN,NaN,NaN
313,NaN,NaN,NaN,NaN
314,NaN,NaN,NaN,NaN
...,...,...,...,...
65529,NaN,NaN,NaN,NaN
65530,NaN,NaN,NaN,NaN
65531,NaN,NaN,NaN,NaN
65532,NaN,NaN,NaN,NaN


In [178]:
crime_raw_data=crime_raw_data[crime_raw_data["죄종"].notnull()]

- 새로운 로우 데이터를 보면 정상적으로 보임
  - integer로 보임

In [179]:
crime_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310 entries, 0 to 309
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      310 non-null    object 
 1   죄종      310 non-null    object 
 2   발생검거    310 non-null    object 
 3   건수      310 non-null    float64
dtypes: float64(1), object(3)
memory usage: 12.1+ KB


# 3. Pandas Pivot table
- index, coloumn, values, aggfunc로 사용 함

In [180]:
df = pd.DataFrame()
df=pd.read_excel("../../data/02.Seoul_Crime/02. sales-funnel.xlsx", engine="openpyxl")
if "Dongmin" in hostname:
    pass
    #df=pd.read_excel("../../data/02.Seoul_Crime/02. sales-funnel.xlsx", engine="openpyxl")
else :
    df=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/data/02.Seoul_Crime/02. sales-funnel.xlsx")

df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


#### values 설정

In [181]:
df.pivot_table(index=["Manager", "Rep"], values="Price")

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

#### aggfunc 를 통해 계산된 컬럼 출력

In [182]:
df.pivot_table(index=["Manager", "Rep"], values="Price",aggfunc=[np.sum,len])

sum   len
                              Price Price
Manager       Rep                        
Debra Henley  Craig Booker    80000     4
              Daniel Hilton  115000     3
              John Smith      40000     2
Fred Anderson Cedric Moss    110000     4
              Wendy Yule     177000     4

#### columns 설정

In [183]:
df.pivot_table(index=["Manager", "Rep"], values="Price",columns="Product",aggfunc=[np.sum,len])

sum                               len  \
Product                           CPU Maintenance Monitor Software  CPU   
Manager       Rep                                                         
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0  2.0   
              Daniel Hilton  105000.0         NaN     NaN  10000.0  2.0   
              John Smith      35000.0      5000.0     NaN      NaN  1.0   
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0  2.0   
              Wendy Yule     165000.0      7000.0  5000.0      NaN  2.0   

                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker          1.0     NaN      1.0  
              Daniel Hilton         NaN     NaN      1.0  
              John Smith            1.0     NaN      NaN  
Fred Anderson Cedric Moss           1.0     NaN      1.0  
              Wendy Yule            1.0     1.0      NaN

#### NaN값 설정 : fill valuem

In [184]:
df.pivot_table(index=["Manager", "Rep"], values="Price",columns="Product",aggfunc=[np.sum,len],fill_value=0)

sum                              len  \
Product                         CPU Maintenance Monitor Software CPU   
Manager       Rep                                                      
Debra Henley  Craig Booker    65000        5000       0    10000   2   
              Daniel Hilton  105000           0       0    10000   2   
              John Smith      35000        5000       0        0   1   
Fred Anderson Cedric Moss     95000        5000       0    10000   2   
              Wendy Yule     165000        7000    5000        0   2   

                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            1       0        1  
              Daniel Hilton           0       0        1  
              John Smith              1       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              1       1        0

#### 2개 이상의 index와 values 설정

In [185]:
df.pivot_table(index=["Manager", "Rep","Product"], values=["Price","Quantity"],aggfunc=[np.sum,len],fill_value=0)

sum            len         
                                          Price Quantity Price Quantity
Manager       Rep           Product                                    
Debra Henley  Craig Booker  CPU           65000        2     2        2
                            Maintenance    5000        2     1        1
                            Software      10000        1     1        1
              Daniel Hilton CPU          105000        4     2        2
                            Software      10000        1     1        1
              John Smith    CPU           35000        1     1        1
                            Maintenance    5000        2     1        1
Fred Anderson Cedric Moss   CPU           95000        3     2        2
                            Maintenance    5000        1     1        1
                            Software      10000        1     1        1
              Wendy Yule    CPU          165000        7     2        2
                            Maintenance    7000        3     1        1
                            Monitor        5000        2     1        1

#### aggfunc 2개 이상 설정

In [186]:
df.pivot_table(
    index=["Manager", "Rep", "Product"],
    values=["Price", "Quantity"],
    aggfunc=[np.sum, np.mean],
    fill_value=0,
    margins=True
)

sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

# 4. 서울시 범죄 현황 데이터 정리

- pivot table로 변경

In [187]:
crime_station = crime_raw_data.pivot_table(
    crime_raw_data,
    index="구분",
    columns=["죄종","발생검거"],
    aggfunc=[np.sum]
)
crime_station.head()

sum                                                              
         건수                                                              
죄종       강간           강도          살인           절도              폭력        
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생
구분                                                                       
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0

- column 데이터 확인

In [188]:
crime_station.columns

MultiIndex([('sum', '건수', '강간', '검거'),
            ('sum', '건수', '강간', '발생'),
            ('sum', '건수', '강도', '검거'),
            ('sum', '건수', '강도', '발생'),
            ('sum', '건수', '살인', '검거'),
            ('sum', '건수', '살인', '발생'),
            ('sum', '건수', '절도', '검거'),
            ('sum', '건수', '절도', '발생'),
            ('sum', '건수', '폭력', '검거'),
            ('sum', '건수', '폭력', '발생')],
           names=[None, None, '죄종', '발생검거'])

- 멀티 인덱스에서 하나의 데이터에 접근하기
- 시리즈로 반환 받음

In [189]:
crime_station['sum', '건수', '폭력', '발생']

구분
강남     2336.0
강동     2530.0
강북     2778.0
강서     3204.0
관악     3235.0
광진     2392.0
구로     3164.0
금천     2193.0
남대문     890.0
노원     2516.0
도봉     1441.0
동대문    2642.0
동작     1661.0
마포     2939.0
방배      500.0
서대문    1976.0
서부     1128.0
서초     1838.0
성동     1619.0
성북     1176.0
송파     3289.0
수서     1666.0
양천     2250.0
영등포    3593.0
용산     2384.0
은평     1536.0
종로     1291.0
종암      932.0
중랑     3164.0
중부     1335.0
혜화      972.0
Name: (sum, 건수, 폭력, 발생), dtype: float64

- droplevel : 다중 컬럼에서 특정컬럼 제거

In [190]:
crime_station.columns = crime_station.columns.droplevel([0,1])

In [191]:
crime_station.columns

MultiIndex([('강간', '검거'),
            ('강간', '발생'),
            ('강도', '검거'),
            ('강도', '발생'),
            ('살인', '검거'),
            ('살인', '발생'),
            ('절도', '검거'),
            ('절도', '발생'),
            ('폭력', '검거'),
            ('폭력', '발생')],
           names=['죄종', '발생검거'])

In [192]:
crime_station.head()

죄종       강간           강도          살인           절도              폭력        
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생
구분                                                                       
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0

- 현재 index= 경찰서 이름
- 경찰서 이름을 통해 지역구 이름 유추해야함

In [193]:
crime_station.index

Index(['강남', '강동', '강북', '강서', '관악', '광진', '구로', '금천', '남대문', '노원', '도봉',
       '동대문', '동작', '마포', '방배', '서대문', '서부', '서초', '성동', '성북', '송파', '수서',
       '양천', '영등포', '용산', '은평', '종로', '종암', '중랑', '중부', '혜화'],
      dtype='object', name='구분')

# 4. Python 모듈 설치

## pip
- 파이썬 공식 모듈 관리자
- 명령어 :
  - pip list : 현재 설치된 모듈 반환
  - pip install 모듈명 : 모듈 설치
  - pip uninstall 모듈명 : 모듈 삭제

## conda
- pip는 콘다 환경에서 의존성 관리가 정확하지 않음
- 다른 명령어는 pip와 동일
- conda install -c 채널명 모듈 : 해당 채널에 모듈 설치
- 단, 모든 모듈이 콘다로 설치되는 것이 아님


In [194]:
get_ipython().system("pip list")

Package                   Version
------------------------- -----------
anyio                     4.2.0
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
asttokens                 2.0.5
async-lru                 2.0.4
attrs                     24.2.0
Babel                     2.11.0
backcall                  0.2.0
beautifulsoup4            4.12.3
bleach                    4.1.0
Bottleneck                1.3.7
Brotli                    1.0.9
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.3.2
colorama                  0.4.6
comm                      0.2.1
contourpy                 1.0.5
cycler                    0.11.0
debugpy                   1.6.7
decorator                 5.1.1
defusedxml                0.7.1
et-xmlfile                1.1.0
exceptiongroup            1.2.0
executing                 0.8.3
fastjsonschema            2.16.2
fonttools                 4.51.0
googlemaps                4.10.0
h11               

# 5. Google Mpas API 이용

In [195]:
import googlemaps

## 1) Google Maps API 설치

In [196]:
import json
import os

current_dir = os.getcwd()
config_dir = os.path.join(current_dir, '../../data/')

config_file_path = os.path.join(config_dir, 'config.json')
with open(config_file_path, "r") as f:
    config = json.load(f)
gmaps_key = config["gcp"]["api_key"]

In [197]:
gmaps = googlemaps.Client(key=gmaps_key)

In [198]:
gmaps.geocode("서울영등포경찰서",language="ko")

[{'address_components': [{'long_name': '608',
    'short_name': '608',
    'types': ['premise']},
   {'long_name': '국회대로',
    'short_name': '국회대로',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': '영등포구',
    'short_name': '영등포구',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': '서울특별시',
    'short_name': '서울특별시',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '150-043',
    'short_name': '150-043',
    'types': ['postal_code']}],
  'formatted_address': '대한민국 서울특별시 영등포구 국회대로 608',
  'geometry': {'location': {'lat': 37.5260441, 'lng': 126.9008091},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 37.5273930802915,
     'lng': 126.9021580802915},
    'southwest': {'lat': 37.5246951197085, 'lng': 126.8994601197085}}},
  'place_id': 'ChIJ1TimJLaffDURptXOs0Tj6sY',
  'plus_code': {'comp

## Python 반목문

###  for 문  예제

In [199]:
for i in [1,2,3,4]:
    print("숫자는 : ",i)

숫자는 :  1
숫자는 :  2
숫자는 :  3
숫자는 :  4


In [200]:
for i in range(0,10):
    print( i ** 2)

0
1
4
9
16
25
36
49
64
81


#### list comprehension

In [201]:
[n**2 for n in range(0,10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

### Pandas에 맞춰진 반복문용 명령 iterrows()
- Pandas 데이터 프레임은 대부분 2차원
    - for문을 사용하면 n번째라는 지정을 반복 > 가독률이 떨어짐
- Pandas 데이터 프레임으로 반복문을 만들때 iterrows() 옵션을 사용하는게 좋음
- 받을 때, 인덱스와 내용으로 나누어 받는 차이가 있음 m

## 2) Google Maps를 이용해 데이터 정리

- API 동작 테스트

In [202]:
gmap = googlemaps.Client(key=gmaps_key)
tmp=gmaps.geocode("서울영등포경찰서",language="ko")

### 필요한 데이터만 수집
- formatted_address
- geometry : 위도 경도 정보

In [203]:
###위도
print(tmp[0].get("geometry")["location"]["lat"])
### 경도
print(tmp[0].get("geometry")["location"]["lng"])

37.5260441
126.9008091


In [204]:
### 주소 출력 - 지역구만
tmp[0].get("formatted_address").split()[2]

'영등포구'

- 구별 데이터 + 위도 경도  컬럼 추가

In [205]:
### np.nan : NaN값으로 채움
crime_station["구별"] = np.nan
crime_station["lat"] = np.nan
crime_station["lng"] = np.nan
crime_station.head()

죄종       강간           강도          살인           절도              폭력          구별  \
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생       
구분                                                                              
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0 NaN   
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0 NaN   
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0 NaN   
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0 NaN   
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0 NaN   

죄종   lat lng  
발생검거          
구분            
강남   NaN NaN  
강동   NaN NaN  
강북   NaN NaN  
강서   NaN NaN  
관악   NaN NaN

- 경찰서 이름에서 소속된 구이름 찾기
- 구 이름과 위도 경도 정보 저장
반복문을 이용해 NaN을 채움

In [206]:
for idx,rows in crime_station.iterrows():
    ### 경찰서 이름으로 만듦
    station_name = "서울" + str(idx) + "경찰서"
    tmp = gmaps.geocode(station_name, language="ko")
    tmp[0].get("formatted_address")
    tmp_gu=tmp[0].get("formatted_address")
    lat = tmp[0].get("geometry")["location"]["lat"]
    lng = tmp[0].get("geometry")["location"]["lng"]

    ### idx번째 lat에 위도 데이터 추가
    crime_station.loc[idx, "lat"] =lat
    crime_station.loc[idx, "lng"] =lng
    crime_station.loc[idx, "구별"]=tmp_gu.split()[2]


In [207]:
crime_station.head()

죄종       강간           강도          살인           절도              폭력          \
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생   
구분                                                                          
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0   
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0   
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0   
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0   
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0   

죄종     구별        lat         lng  
발생검거                              
구분                                
강남    강남구  37.509435  127.066958  
강동    강동구  37.528511  127.126822  
강북    강북구  37.637197  127.027305  
강서    강서구  37.551362  126.850280  
관악    관악구  37.474395  126.951349

In [208]:
### 한줄씩 보기
crime_station.columns.get_level_values(0)

Index(['강간', '강간', '강도', '강도', '살인', '살인', '절도', '절도', '폭력', '폭력', '구별', 'lat',
       'lng'],
      dtype='object', name='죄종')

In [209]:
### 한줄씩 보기
crime_station.columns.get_level_values(1)[2]

'검거'

In [210]:
### 컬럼 갯수 세기
len(crime_station.columns.get_level_values(0))

13

### 컬럼 내용 정리

In [211]:
tmp = [
    crime_station.columns.get_level_values(0)[n] + crime_station.columns.get_level_values(1)[n]
    for n in range(0,len(crime_station.columns.get_level_values(0)))
]

tmp

['강간검거',
 '강간발생',
 '강도검거',
 '강도발생',
 '살인검거',
 '살인발생',
 '절도검거',
 '절도발생',
 '폭력검거',
 '폭력발생',
 '구별',
 'lat',
 'lng']

In [212]:
crime_station.columns = tmp
crime_station.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,구별,lat,lng
구분,,,,,,,,,,,,,
강남,269.0,339.0,26.0,24.0,3.0,3.0,1129.0,2438.0,2096.0,2336.0,강남구,37.509435,127.066958
강동,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,강동구,37.528511,127.126822
강북,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,강북구,37.637197,127.027305
강서,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0,강서구,37.551362,126.850280
관악,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0,관악구,37.474395,126.951349


In [213]:
### 데이터 저장
crime_station.to_csv("../../data/02.Seoul_Crime/02. crime_in_Seoul_raw.csv",sep=",",encoding="utf-8")

In [214]:
pd.read_csv("../../data/02.Seoul_Crime/02. crime_in_Seoul_raw.csv")

,구분,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,구별,lat,lng
0,강남,269.0,339.0,26.0,24.0,3.0,3.0,1129.0,2438.0,2096.0,2336.0,강남구,37.509435,127.066958
1,강동,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,강동구,37.528511,127.126822
2,강북,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,강북구,37.637197,127.027305
3,강서,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0,강서구,37.551362,126.850280
4,관악,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0,관악구,37.474395,126.951349
5,광진,234.0,279.0,6.0,11.0,4.0,4.0,1057.0,2636.0,2011.0,2392.0,광진구,37.542823,127.083839
6,구로,181.0,273.0,13.0,10.0,9.0,9.0,861.0,1910.0,2680.0,3164.0,구로구,37.507442,126.890224
7,금천,143.0,175.0,7.0,7.0,6.0,6.0,654.0,1264.0,1946.0,2193.0,금천구,37.456813,126.896806
8,남대문,52.0,57.0,4.0,5.0,1.0,1.0,429.0,946.0,832.0,890.0,중구,37.554758,126.973498
9,노원,142.0,159.0,9.0,6.0,6.0,5.0,740.0,1857.0,2124.0,2516.0,노원구,37.642139,127.071047


b# 6. 구별 데이터 정리

- index_col : "구분"을 인덱스 컬럼으로 사용

In [215]:
crime_anal_station=pd.read_csv("../../data/02.Seoul_Crime/02. crime_in_Seoul_raw.csv",index_col=0, encoding="utf-8")
crime_anal_station.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,구별,lat,lng
구분,,,,,,,,,,,,,
강남,269.0,339.0,26.0,24.0,3.0,3.0,1129.0,2438.0,2096.0,2336.0,강남구,37.509435,127.066958
강동,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,강동구,37.528511,127.126822
강북,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,강북구,37.637197,127.027305
강서,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0,강서구,37.551362,126.850280
관악,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0,관악구,37.474395,126.951349


In [216]:
crime_anal_gu = pd.pivot_table(crime_anal_station,index="구별",aggfunc=np.sum)
del crime_anal_gu["lat"]
crime_anal_gu.drop("lng",axis=1,inplace=True)
crime_anal_gu.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생
구별,,,,,,,,,,
강남구,413.0,516.0,42.0,39.0,5.0,5.0,1918.0,3587.0,3527.0,4002.0
강동구,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0
강북구,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0
강서구,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0
관악구,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0


### 검거율 생성
- 하나의 컬럼을 다른 컬럼으로 나누기 m

In [217]:
crime_anal_gu["강도검거"]/crime_anal_gu["강도발생"]

구별
강남구     1.076923
강동구     0.928571
강북구     0.800000
강서구     1.000000
관악구     0.833333
광진구     0.545455
구로구     1.300000
금천구     1.000000
노원구     1.500000
도봉구     1.000000
동대문구    1.200000
동작구     1.000000
마포구     1.750000
서대문구    0.800000
서초구     0.769231
성동구     1.666667
성북구     1.000000
송파구     0.800000
양천구     1.000000
영등포구    0.736842
용산구     1.111111
은평구     0.777778
종로구     0.750000
중구      0.875000
중랑구     1.000000
dtype: float64

In [218]:
### 다수의 컬럼을 다른 컬럼으로 나누기
crime_anal_gu[["강도검거","살인검거"]].div(crime_anal_gu["강도발생"],axis=0).head(3)

,강도검거,살인검거
구별,,
강남구,1.076923,0.128205
강동구,0.928571,0.357143
강북구,0.800000,1.200000


In [219]:
### 다수의 컬럼을 다수의 컬럼으로 나누기
num = ["강간검거","강도검거","살인검거","절도검거","폭력검거"]
den = ["강간발생","강도발생","살인발생","절도발생","폭력발생"]

crime_anal_gu[num].div(crime_anal_gu[den].values).head()

,강간검거,강도검거,살인검거,절도검거,폭력검거
구별,,,,,
강남구,0.800388,1.076923,1.000000,0.534709,0.881309
강동구,0.950000,0.928571,1.250000,0.514253,0.869960
강북구,0.732719,0.800000,0.857143,0.549918,0.893449
강서구,0.869091,1.000000,1.111111,0.548156,0.863920
관악구,0.819876,0.833333,1.166667,0.445554,0.836785


In [220]:
### 검거율 컬럼 만들기
target = ["강간검거율","강도검거율","살인검거율","절도검거율","폭력검거율"]

num = ["강간검거","강도검거","살인검거","절도검거","폭력검거"]
den = ["강간발생","강도발생","살인발생","절도발생","폭력발생"]

### 컬럼 생성
crime_anal_gu[target]=crime_anal_gu[num].div(crime_anal_gu[den].values)*100
crime_anal_gu.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,,,,,,
강남구,413.0,516.0,42.0,39.0,5.0,5.0,1918.0,3587.0,3527.0,4002.0,80.038760,107.692308,100.000000,53.470867,88.130935
강동구,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,95.000000,92.857143,125.000000,51.425314,86.996047
강북구,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
강서구,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0,86.909091,100.000000,111.111111,54.815574,86.392010
관악구,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0,81.987578,83.333333,116.666667,44.555397,83.678516


- 필요없는 컬럼 제거하기

In [221]:
#del crime_anal_gu["강간검거"]
#del crime_anal_gu["강도검거"]
#del crime_anal_gu["살인검거"]
#del crime_anal_gu["절도검거"]
#del crime_anal_gu["폭력검거"]

crime_anal_gu.drop(["강간검거","강도검거","살인검거","절도검거","폭력검거"],axis=1,inplace=True)
crime_anal_gu.head()

,강간발생,강도발생,살인발생,절도발생,폭력발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,516.0,39.0,5.0,3587.0,4002.0,80.038760,107.692308,100.000000,53.470867,88.130935
강동구,160.0,14.0,4.0,1754.0,2530.0,95.000000,92.857143,125.000000,51.425314,86.996047
강북구,217.0,5.0,7.0,1222.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
강서구,275.0,10.0,9.0,1952.0,3204.0,86.909091,100.000000,111.111111,54.815574,86.392010
관악구,322.0,12.0,6.0,2103.0,3235.0,81.987578,83.333333,116.666667,44.555397,83.678516


- 100보다 큰 숫자를 바꾸기

In [222]:
crime_anal_gu[crime_anal_gu[target] > 100] = 100
crime_anal_gu.head()

,강간발생,강도발생,살인발생,절도발생,폭력발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,516.0,39.0,5.0,3587.0,4002.0,80.038760,100.000000,100.000000,53.470867,88.130935
강동구,160.0,14.0,4.0,1754.0,2530.0,95.000000,92.857143,100.000000,51.425314,86.996047
강북구,217.0,5.0,7.0,1222.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
강서구,275.0,10.0,9.0,1952.0,3204.0,86.909091,100.000000,100.000000,54.815574,86.392010
관악구,322.0,12.0,6.0,2103.0,3235.0,81.987578,83.333333,100.000000,44.555397,83.678516


In [223]:
### 컬럼이름 변경
crime_anal_gu.rename(columns={"강간발생":"강간","강도발생":"강도","살인발생":"살인","절도발생":"절도","폭력발생":"폭력"},inplace=True)

In [224]:
crime_anal_gu.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,516.0,39.0,5.0,3587.0,4002.0,80.038760,100.000000,100.000000,53.470867,88.130935
강동구,160.0,14.0,4.0,1754.0,2530.0,95.000000,92.857143,100.000000,51.425314,86.996047
강북구,217.0,5.0,7.0,1222.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
강서구,275.0,10.0,9.0,1952.0,3204.0,86.909091,100.000000,100.000000,54.815574,86.392010
관악구,322.0,12.0,6.0,2103.0,3235.0,81.987578,83.333333,100.000000,44.555397,83.678516


# 7. 범죄 데이터 정렬을 위한 데이터 정리

In [225]:
crime_anal_gu.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,516.0,39.0,5.0,3587.0,4002.0,80.038760,100.000000,100.000000,53.470867,88.130935
강동구,160.0,14.0,4.0,1754.0,2530.0,95.000000,92.857143,100.000000,51.425314,86.996047
강북구,217.0,5.0,7.0,1222.0,2778.0,73.271889,80.000000,85.714286,54.991817,89.344852
강서구,275.0,10.0,9.0,1952.0,3204.0,86.909091,100.000000,100.000000,54.815574,86.392010
관악구,322.0,12.0,6.0,2103.0,3235.0,81.987578,83.333333,100.000000,44.555397,83.678516


- 정규화 : 최고 값은 1 최소 값은 0 으로 변경

In [226]:
### 정규화 예시
crime_anal_gu["강도"] / crime_anal_gu["강도"].max()

구별
강남구     1.000000
강동구     0.358974
강북구     0.128205
강서구     0.256410
관악구     0.307692
광진구     0.282051
구로구     0.256410
금천구     0.179487
노원구     0.153846
도봉구     0.128205
동대문구    0.256410
동작구     0.179487
마포구     0.102564
서대문구    0.128205
서초구     0.333333
성동구     0.076923
성북구     0.205128
송파구     0.384615
양천구     0.179487
영등포구    0.487179
용산구     0.230769
은평구     0.230769
종로구     0.307692
중구      0.205128
중랑구     0.358974
Name: 강도, dtype: float64

In [234]:
col  = ["살인","강도","강간","절도","폭력"]
crime_anal_norm[col] = crime_anal_gu[col]/crime_anal_gu[col].max()
crime_anal_norm.head()

,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,살인,강도,강간,절도,폭력
구별,,,,,,,,,,
강남구,1.000000,1.000000,0.813617,0.773418,0.970181,0.384615,1.000000,1.000000,1.000000,1.000000
강동구,1.000000,0.928571,0.965702,0.743830,0.957688,0.307692,0.358974,0.310078,0.488988,0.632184
강북구,0.857143,0.800000,0.744830,0.795417,0.983544,0.538462,0.128205,0.420543,0.340675,0.694153
강서구,1.000000,1.000000,0.883456,0.792868,0.951038,0.692308,0.256410,0.532946,0.544187,0.800600
관악구,1.000000,0.833333,0.833427,0.644462,0.921167,0.461538,0.307692,0.624031,0.586284,0.808346


In [235]:
### 검거율 데이터 추가
col2  = ["살인검거율","강도검거율","강간검거율","절도검거율","폭력검거율"]
crime_anal_norm[col2] = crime_anal_gu[col2]/crime_anal_gu[col2].max()
crime_anal_norm.head()

,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,살인,강도,강간,절도,폭력
구별,,,,,,,,,,
강남구,1.000000,1.000000,0.813617,0.773418,0.970181,0.384615,1.000000,1.000000,1.000000,1.000000
강동구,1.000000,0.928571,0.965702,0.743830,0.957688,0.307692,0.358974,0.310078,0.488988,0.632184
강북구,0.857143,0.800000,0.744830,0.795417,0.983544,0.538462,0.128205,0.420543,0.340675,0.694153
강서구,1.000000,1.000000,0.883456,0.792868,0.951038,0.692308,0.256410,0.532946,0.544187,0.800600
관악구,1.000000,0.833333,0.833427,0.644462,0.921167,0.461538,0.307692,0.624031,0.586284,0.808346


In [229]:
### 구별 CCTV 자료의 인구수와 CCTV수 추가
result_CCTV = pd.read_csv("../../data/01.Seoul_CCTV/01.result.csv" ,index_col="구별", encoding="utf-8")

crime_anal_norm[["인구수", "CCTV"]] = result_CCTV[["인구수", "소계"]]
crime_anal_norm.head()

,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,인구수,CCTV
구별,,,,,,,
강남구,1.000000,1.000000,0.813617,0.773418,0.970181,561052,3238
강동구,1.000000,0.928571,0.965702,0.743830,0.957688,440359,1010
강북구,0.857143,0.800000,0.744830,0.795417,0.983544,328002,831
강서구,1.000000,1.000000,0.883456,0.792868,0.951038,608255,911
관악구,1.000000,0.833333,0.833427,0.644462,0.921167,520929,2109


- 정규화된 범죄 발생 건수 전체의 평균을 구해서 범죄 컬럼 대표값으로 사용

In [236]:
col = ["강간", "강도", "살인", "절도", "폭력"]
crime_anal_norm["범죄"] = np.mean(crime_anal_norm[col], axis=1)
crime_anal_norm.head()

,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,살인,강도,강간,절도,폭력,범죄
구별,,,,,,,,,,,
강남구,1.000000,1.000000,0.813617,0.773418,0.970181,0.384615,1.000000,1.000000,1.000000,1.000000,0.876923
강동구,1.000000,0.928571,0.965702,0.743830,0.957688,0.307692,0.358974,0.310078,0.488988,0.632184,0.419583
강북구,0.857143,0.800000,0.744830,0.795417,0.983544,0.538462,0.128205,0.420543,0.340675,0.694153,0.424407
강서구,1.000000,1.000000,0.883456,0.792868,0.951038,0.692308,0.256410,0.532946,0.544187,0.800600,0.565290
관악구,1.000000,0.833333,0.833427,0.644462,0.921167,0.461538,0.307692,0.624031,0.586284,0.808346,0.557578


In [238]:
### numpy.mean 예시
np.mean(np.array(
    [[0.357143, 1.000000, 1.000000, 0.977118, 0.733773],
    [0.285714, 0.358974, 0.310078, 0.477799, 0.463880]]
),axis=1) #axis=1 행

array([0.8136068, 0.379289 ])

### 검거율 평균 구하기

In [240]:
col = ["강간검거율", "강도검거율", "살인검거율", "절도검거율", "폭력검거율"]
crime_anal_norm["검거"] = np.mean(crime_anal_norm[col], axis=1) # axis=1 행을 따라서 연산하는 옵션
crime_anal_norm.head()

,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,살인,강도,강간,절도,폭력,범죄,검거
구별,,,,,,,,,,,,
강남구,1.000000,1.000000,0.813617,0.773418,0.970181,0.384615,1.000000,1.000000,1.000000,1.000000,0.876923,0.911443
강동구,1.000000,0.928571,0.965702,0.743830,0.957688,0.307692,0.358974,0.310078,0.488988,0.632184,0.419583,0.919158
강북구,0.857143,0.800000,0.744830,0.795417,0.983544,0.538462,0.128205,0.420543,0.340675,0.694153,0.424407,0.836187
강서구,1.000000,1.000000,0.883456,0.792868,0.951038,0.692308,0.256410,0.532946,0.544187,0.800600,0.565290,0.925472
관악구,1.000000,0.833333,0.833427,0.644462,0.921167,0.461538,0.307692,0.624031,0.586284,0.808346,0.557578,0.846478
